In [71]:
import pandas as pd
from zipfile import Path as ZipPath
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import matplotlib as mpl
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    "text.usetex": True,
    "font.family": "serif",
})
import matplotlib.pyplot as plt


# wczytywanie bezpośrednio z archiwum

In [72]:
fps = {x.stem: x for x in ZipPath('Dopasowanie.zip', at='/').iterdir()}
fps                                                                

{'IRL-2.2': Path('Dopasowanie.zip', 'IRL-2.2.csv'),
 'IRL-2.1': Path('Dopasowanie.zip', 'IRL-2.1.csv'),
 'trpp': Path('Dopasowanie.zip', 'trpp.csv')}

In [73]:
for k,v in fps.items():
    print(k, v.exists())

IRL-2.2 True
IRL-2.1 True
trpp True


In [74]:
%%time

dfs = {}
for k,v in fps.items():
    with v.open() as f:
        dfs[k] = pd.read_csv(f, parse_dates=['UTC'], index_col='UTC')


CPU times: user 1.27 s, sys: 16 ms, total: 1.28 s
Wall time: 1.29 s


# połączenie w jednego dataframe


In [75]:
df = pd.concat(
    [
        dfs["IRL-2.1"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.1"}),
        dfs["IRL-2.2"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.2"}),
        dfs["trpp"]
    ],
    axis=1
).sort_index()


In [76]:
print(df.columns.tolist())
print(dfs.keys())

['IRL-2.1', 'IRL-2.2', 'PLC_time', 'PAR', 'PK1', 'PK2', 'PK3', 'PK4', 'PK5', 'PK6', 'PB1', 'PB2', 'PB3', 'PB4', 'PB5', 'PB6']
dict_keys(['IRL-2.2', 'IRL-2.1', 'trpp'])


In [77]:

%%time
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

# GÓRA: IRL (tylko raz)
df.plot(
    ax=ax1,
    y=["IRL-2.1", "IRL-2.2"],
    logy=True,
    grid=True,
    marker=".",
)

# DÓŁ: TRPP
df.plot(
    ax=ax2,
    y=["PAR", "PK2", "PK6", "PB1"],
    grid=True,
    marker=".",
)

# ax1.legend()
# ax2.legend()
# plt.tight_layout()
# fig.savefig(f"{outdir}/całośc.png", format="png")  # bez tight

# ax1.legend()


CPU times: user 723 ms, sys: 23.8 ms, total: 747 ms
Wall time: 754 ms


<Axes: xlabel='UTC'>

In [78]:
intervals = {
    "zrzuty": slice("2025-12-08 19:55:30", "2025-12-08 21:30:30"),
    "zrzut01": slice("2025-12-08 19:55:30", "2025-12-08 20:03:30"),
    "zrzut02": slice("2025-12-08 20:07:30", "2025-12-08 20:11:30"),
    "zrzut03": slice("2025-12-08 20:15:30", "2025-12-08 20:19:30"),
    "zrzut04": slice("2025-12-08 20:23:30", "2025-12-08 20:27:30"),
    "zrzut05": slice("2025-12-08 20:31:30", "2025-12-08 20:35:30"),
    "zrzut06": slice("2025-12-08 20:39:30", "2025-12-08 20:43:30"),
    "zrzut07": slice("2025-12-08 20:47:30", "2025-12-08 20:51:30"),
    "zrzut08": slice("2025-12-08 20:55:30", "2025-12-08 20:59:30"),
    "zrzut09": slice("2025-12-08 21:00:30", "2025-12-08 21:07:30"),
    "zrzut10": slice("2025-12-08 21:08:30", "2025-12-08 21:12:30"),
    "zrzut11": slice("2025-12-08 21:12:30", "2025-12-08 21:18:30"),
}

[(k, v.start, v.stop) for k, v in intervals.items()]


[('zrzuty', '2025-12-08 19:55:30', '2025-12-08 21:30:30'),
 ('zrzut01', '2025-12-08 19:55:30', '2025-12-08 20:03:30'),
 ('zrzut02', '2025-12-08 20:07:30', '2025-12-08 20:11:30'),
 ('zrzut03', '2025-12-08 20:15:30', '2025-12-08 20:19:30'),
 ('zrzut04', '2025-12-08 20:23:30', '2025-12-08 20:27:30'),
 ('zrzut05', '2025-12-08 20:31:30', '2025-12-08 20:35:30'),
 ('zrzut06', '2025-12-08 20:39:30', '2025-12-08 20:43:30'),
 ('zrzut07', '2025-12-08 20:47:30', '2025-12-08 20:51:30'),
 ('zrzut08', '2025-12-08 20:55:30', '2025-12-08 20:59:30'),
 ('zrzut09', '2025-12-08 21:00:30', '2025-12-08 21:07:30'),
 ('zrzut10', '2025-12-08 21:08:30', '2025-12-08 21:12:30'),
 ('zrzut11', '2025-12-08 21:12:30', '2025-12-08 21:18:30')]

In [79]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

for name, sl in intervals.items():

    df_cut = df.loc[sl]
    if name == "zrzuty":
        continue
    if df_cut.empty:
        print(f"{name}: pusty zakres")
        continue

    fig, (ax1, ax2) = plt.subplots(
        nrows=2,
        figsize=(16, 12),
        sharex=True
    )

    # ===== GÓRA: IRL =====
    df_cut.plot(
        ax=ax1,
        y=["IRL-2.2"],
        logy=True,
        grid=True,
        marker=".",
    )

    # ===== DÓŁ: PK6 =====
    df_cut.plot(
        ax=ax2,
        y=["PK6"],
        grid=True,
        marker=".",
    )

    # format osi X
    ax1.set_title(f"{name} | {sl.start} → {sl.stop}")
    ax2.set_xlabel("UTC")

    plt.tight_layout()
    plt.show()



/tmp/ipykernel_42878/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_42878/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_42878/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_42878/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_42878/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_42878/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_42878/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_42878/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [80]:
df_cut = df.loc[intervals["zrzut01"]]

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

df_cut.plot(
    ax=ax1,
    y=["IRL-2.2"],
    logy=True,
    grid=True,
    marker=".",
)

df_cut.plot(
    ax=ax2,
    y=["PK6"],
    grid=True,
    marker=".",
)

plt.tight_layout()
plt.savefig 
plt.show()


/tmp/ipykernel_42878/400118637.py:22: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [81]:
df_cut = df.loc[intervals["zrzut01"]]

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

df_cut.plot(
    ax=ax1,
    y=["IRL-2.2"],
    logy=True,
    grid=True,
    marker=".",
)

df_cut.plot(
    ax=ax2,
    y=["PK6"],
    grid=True,
    marker=".",
)

plt.tight_layout()
plt.show()


/tmp/ipykernel_42878/4119963235.py:21: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [82]:
cuts = {
    "zrzut01": ("2025-12-08 19:57:30", "2025-12-08 20:01:30"),
   
}

for name, (t0, t1) in cuts.items():
    maly_df = df.loc[t0:t1]

    t = (maly_df.index - maly_df.index[0]).total_seconds()
    y = maly_df["IRL-2.2"]

In [83]:
cuts = {
    "zrzut01": ("2025-12-08 19:57:30", "2025-12-08 20:01:30"),
    "zrzut02": ("2025-12-08 20:08:40", "2025-12-08 20:09:45"),
    "zrzut03": ("2025-12-08 20:17:15", "2025-12-08 20:18:05"),
    "zrzut04": ("2025-12-08 20:25:05", "2025-12-08 20:26:10"),
    "zrzut05": ("2025-12-08 20:32:48", "2025-12-08 20:33:43"),
    "zrzut06": ("2025-12-08 20:41:08", "2025-12-08 20:41:53"),
    "zrzut07": ("2025-12-08 20:49:13", "2025-12-08 20:50:08"),
    "zrzut08": ("2025-12-08 20:56:33", "2025-12-08 20:57:33"),
    "zrzut09": ("2025-12-08 21:02:50", "2025-12-08 21:03:40"),
    "zrzut10": ("2025-12-08 21:09:22", "2025-12-08 21:10:12"),
    "zrzut11": ("2025-12-08 21:14:59", "2025-12-08 21:15:34"),
}

for name, (t0, t1) in cuts.items():
    maly_df = df.loc[t0:t1]

    t = (maly_df.index - maly_df.index[0]).total_seconds()
    y = maly_df["IRL-2.2"]

In [84]:
t

Index([         0.0,  0.023238897,  0.099303007,  0.119539976,  0.199012995,
        0.220527888,  0.299893856,  0.320500851,  0.400672913,  0.423434973,
       ...
       35.500520945, 35.521129847, 35.600239039,  35.62114501,  35.70023799,
       35.720597983, 35.799998045, 35.820545912, 35.899873972, 35.922618866],
      dtype='float64', name='UTC', length=754)

In [85]:

# def lin_fit(t,x1,x2,a,b,c,T,x):   # funkcja do segmentu liniowego
#     y = a*t+b
#     return y

# def par_fit(t,x1,x2,a,b,c,T,x):    # funkcja do segmentu paraboli
#     d=-2*c*x1
#     f = c*x2**2 + d*x2
#     y = c*t**2 *d*t + f
#     return y

# # def exp_fit(t,x1,x2,a,b,c,T,x):   # funkcja do segmentu exp
# #     d=-2*c*x1
# #     f = c*x2**2 + d*x2
# #     g = (c*x2**2 + d*x2+f)/np.exp(x2/T)
# #     y = np.exp(t/T)*g
# #     return y

# def exp_fit(t, a, T):   # funkcja do segmentu exp
#     y = np.exp(t/T) *a
#     return y



In [86]:
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt

b_i   = np.array([2.43e-4, 1.363e-3, 1.203e-3, 2.605e-3, 8.19e-4, 1.67e-4])
lambda_i = np.array([1.27e-2, 3.17e-2, 1.15e-1, 3.11e-1, 1.40,   3.87   ])  # [1/s]
B = b_i.sum()
L = 146e-6

def exp_fit(t, a, T):
    return np.exp(t/T) * a   

def rho_T(T_hat, L,B, b_i, lambda_i):
    suma = np.sum((b_i) / ((1 /T_hat)+lambda_i))
    nie_suma = 1/(T_hat*B)
    rho = nie_suma * (L+ suma)
    return rho

def sigma_rho_T(sigma_T, L,B, b_i, lambda_i, T_hat):
    suma = np.sum((b_i * lambda_i) / (B * (lambda_i*T_hat + 1)**2))
    nie_suma = L/((T_hat**2)*B)
    sigma_rho = (nie_suma + suma)*sigma_T
    return sigma_rho

def pm(val, err, vfmt="{:.2f}", efmt="{:.2f}"):
    return f"{vfmt.format(val)} ± {efmt.format(err)}"

wyniki = []

fig, ax = plt.subplots(figsize=(20,12))

for name, (t0, t1) in cuts.items():
    maly_df = df.loc[t0:t1, ["IRL-2.2"]].dropna()
    if maly_df.empty:
        print(name, "-> pusty wycinek")
        continue

    t = (maly_df.index - maly_df.index[0]).total_seconds().to_numpy()
    y = maly_df["IRL-2.2"].to_numpy()

    if not (np.isfinite(t).all() and np.isfinite(y).all()):
        print(name, "-> NaN/inf w t lub y")
        continue

    # --- FIT DLA TEGO WYCINKA ---
    try:
        popt, pcov = curve_fit(exp_fit, t, y, p0=[y[0], 60], maxfev=10000)
    except Exception as e:
        print(name, "-> curve_fit error:", e)
        continue

    sigma_a, sigma_T = np.sqrt(np.diag(pcov))
    a_hat, T_hat = popt

    r  = rho_T(T_hat, L, B, b_i, lambda_i)
    dr = sigma_rho_T(sigma_T, L, B, b_i, lambda_i, T_hat)

    wyniki.append([name, T_hat, sigma_T, r, dr])

    ax.plot(t, y, marker='.', ls='None', ms=2, alpha=0.6, label=f"{name} data")
    ax.plot(t, exp_fit(t, *popt), lw=2, label=f"{name} fit")

# format wykresu
ax.set_yscale('log')
ax.grid(True, which='both', alpha=0.3)
ax.legend(ncol=2, fontsize=18)
ax.set_xlabel("Czas od początku zrzutu s")
ax.set_ylabel("IRL-2.1 Flux, cps")

# --- tabela wyników ---
import pandas as pd
tab = pd.DataFrame(wyniki, columns=["zrzut", "T_hat [s]", "sigma_T [s]", "rho", "delta_rho, a_hat"])
print(tab)

tab["T ± ΔT"]   = [pm(T, dT, vfmt="{:.3f}", efmt="{:.3f}") for T, dT in zip(tab["T_hat [s]"], tab["sigma_T [s]"])]
tab["ρ ± Δρ"]   = [pm(r, dr, vfmt="{:.6f}", efmt="{:.6f}") for r, dr in zip(tab["rho"], tab["delta_rho, a_hat"])]

print(tab)

fig, ax = plt.subplots(figsize=(6.3, 3.9), constrained_layout=True)

# ... rysowanie ...




      zrzut   T_hat [s]  sigma_T [s]       rho  delta_rho, a_hat
0   zrzut01  212.178116     0.080149  0.051887          0.000017
1   zrzut02   76.599384     0.120679  0.118497          0.000139
2   zrzut03   61.954112     0.105184  0.138256          0.000167
3   zrzut04   76.469017     0.137211  0.118647          0.000158
4   zrzut05   63.597868     0.150027  0.135699          0.000229
5   zrzut06   63.302709     0.127898  0.136151          0.000196
6   zrzut07   71.933919     0.148625  0.124122          0.000188
7   zrzut08   76.516696     0.123147  0.118592          0.000142
8   zrzut09   71.661953     0.126130  0.124467          0.000161
9   zrzut10   67.595633     0.155262  0.129877          0.000216
10  zrzut11   67.910129     0.155538  0.129441          0.000215
      zrzut   T_hat [s]  sigma_T [s]       rho  delta_rho, a_hat  \
0   zrzut01  212.178116     0.080149  0.051887          0.000017   
1   zrzut02   76.599384     0.120679  0.118497          0.000139   
2   zrzut03   61

In [87]:
import numpy as np
import pandas as pd

def stable_samples(series_mm: pd.Series, dt_s: np.ndarray, v_eps_mm_s: float = 0.05) -> pd.Series:
    """
    Zwraca próbki uznane za 'stabilne' (plateau) na podstawie progowania prędkości |dz/dt|.
    v_eps_mm_s dobierz do swojego sygnału (np. 0.02–0.2 mm/s).
    """
    z = pd.to_numeric(series_mm, errors="coerce").dropna()
    if z.size < 3:
        return z.iloc[0:0]

    z_np = z.to_numpy()
    # prędkość na krokach (wyrównanie długości: pierwszej próbce dajemy v=0)
    dz = np.diff(z_np, prepend=z_np[0])
    v  = np.abs(dz / dt_s)  # mm/s

    return z[v <= v_eps_mm_s]

def uncertainty_z_plateau(
    df: pd.DataFrame,
    cuts: dict,
    pos_col: str,
    *,
    k: float = 2.0,
    v_eps_mm_s: float = 0.05,
    min_stable_samples: int = 20,
    dz_resolution_mm: float | None = None,
    use_typeB: bool = True
) -> pd.DataFrame:
    rows = []

    for name, (t0, t1) in cuts.items():
        s_all = df.loc[t0:t1, pos_col]
        s_all = pd.to_numeric(s_all, errors="coerce").dropna()

        if s_all.size < 3:
            rows.append({"zrzut": name, "z_mm": np.nan, "delta_z_mm": np.nan, "note": "brak danych"})
            continue

        # dt między próbkami w sekundach (dla indeksu czasowego)
        t = s_all.index
        dt = (t.to_series().diff().dt.total_seconds()).fillna(method="bfill").to_numpy()
        dt[dt <= 0] = np.median(dt[dt > 0])  # zabezpieczenie

        s_stable = stable_samples(s_all, dt_s=dt, v_eps_mm_s=v_eps_mm_s)

        if s_stable.size < min_stable_samples:
            # fallback: weź ostatnie 25% próbek (często jest już plateau)
            tail = s_all.iloc[int(0.75 * len(s_all)):]
            s_stable = tail if tail.size >= 3 else s_all

        z_hat = float(s_stable.mean())
        s_std = float(s_stable.std(ddof=1)) if s_stable.size >= 2 else 0.0
        uA = s_std / np.sqrt(s_stable.size) if s_stable.size > 0 else np.nan

        uB = 0.0
        if use_typeB and dz_resolution_mm is not None:
            # u_B = Δ/√12 dla rozdzielczości Δ (krok kwantyzacji)
            uB = float(dz_resolution_mm) / np.sqrt(12)

        uc = float(np.sqrt(uA*uA + uB*uB)) if np.isfinite(uA) else np.nan
        U  = float(k * uc) if np.isfinite(uc) else np.nan

        rows.append({
            "zrzut": name,
            "z_mm": z_hat,
            "delta_z_mm": U,              # to jest "±" (U=k*uc)
            "N_stable": int(s_stable.size),
            "uA_mm": float(uA),
            "uB_mm": float(uB) if (use_typeB and dz_resolution_mm is not None) else np.nan,
            "note": ""
        })

    return pd.DataFrame(rows).sort_values("zrzut")


In [100]:
# Podaj prawdziwą nazwę kolumny z położeniem PK6:
pos_col = "PK6"  # <-- zmień jeśli inaczej

tab_z = uncertainty_z_plateau(
    df, cuts, pos_col,
    v_eps_mm_s=0.05,          # próg "ruchu" — do strojenia
    min_stable_samples=20,
    dz_resolution_mm=0.01,    # jeśli znasz rozdzielczość (np. 0.01 mm). Jak nie znasz -> daj None.
    use_typeB=True,
    k=2.0
)

tab = tab.drop(columns=[c for c in ["z_mm","delta_z_mm"] if c in tab.columns])
tab = tab.merge(tab_z[["zrzut","z_mm","delta_z_mm"]], on="zrzut", how="left")
print(tab)

      zrzut   T_hat [s]  sigma_T [s]       rho  delta_rho, a_hat  \
0   zrzut01  212.178116     0.080149  0.051887          0.000017   
1   zrzut02   76.599384     0.120679  0.118497          0.000139   
2   zrzut03   61.954112     0.105184  0.138256          0.000167   
3   zrzut04   76.469017     0.137211  0.118647          0.000158   
4   zrzut05   63.597868     0.150027  0.135699          0.000229   
5   zrzut06   63.302709     0.127898  0.136151          0.000196   
6   zrzut07   71.933919     0.148625  0.124122          0.000188   
7   zrzut08   76.516696     0.123147  0.118592          0.000142   
8   zrzut09   71.661953     0.126130  0.124467          0.000161   
9   zrzut10   67.595633     0.155262  0.129877          0.000216   
10  zrzut11   67.910129     0.155538  0.129441          0.000215   

             T ± ΔT               ρ ± Δρ         z_mm  delta_z_mm  
0   212.178 ± 0.080  0.051887 ± 0.000017   219.000000    0.005774  
1    76.599 ± 0.121  0.118497 ± 0.000139   307.

/tmp/ipykernel_42878/945940751.py:43: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dt = (t.to_series().diff().dt.total_seconds()).fillna(method="bfill").to_numpy()


In [101]:
for name, (t0, t1) in cuts.items():
    maly_df = df.loc[t0:t1, ["IRL-2.2"]].dropna()
    if maly_df.empty:
        continue

    t = (maly_df.index - maly_df.index[0]).total_seconds().to_numpy()
    y = maly_df["IRL-2.2"].to_numpy()

    popt, pcov = curve_fit(exp_fit, t, y, p0=[y[0], 60], maxfev=10000)

    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(t, y, '.', label='zrzut')
    ax.plot(t, exp_fit(t, *popt), 'r-', label='fit')

    ax.set_title(name)
    ax.set_xlabel("czas s")
    ax.set_ylabel("IRL-2.2 cps")
    ax.set_yscale("log")
    ax.grid(True, which="both", alpha=0.3)
    ax.legend()
    outdir = "/home/piegus/ncbj/Inzynierka/figures/"
    fig.savefig(f"{outdir}/{name}_IRL2.pgf", bbox_inches="tight")




Funkcja: scipy.optimize.curve_fit – nieliniowe MNK (Levenberg–Marquardt domyślnie).
exp_fit – Twoja funkcja modelu 
f(t; a,T) zwracająca wartościdopasowania dla wektora t
t – wektor/seria argumentu niezależnego (np. czas).
y – wektor danych obserwowanych (zmienna zależna).
p0=[y.iloc[0], 60] – zgad startowy parametrów [alfa i T ]
maxfev=10000 – maks. liczba wywołań funkcji modelu (zwiększa szansę zbieżności).

In [102]:
popt, pcov = curve_fit(exp_fit, t,y, p0=[y[0], 60], maxfev=10000)
popt, np.sqrt(np.diag(pcov)) # pierwszy to parametry a i T drugi to ich niepewności

(array([26383.68778465,    67.91012883]), array([20.79476392,  0.15553792]))

popt – wektor estymowanych parametrów
pcov – przybliżona macierz kowariancji estymatorów (Podaje nam wartość z moich danych odchylenia standardowego, ale zwraca jest jako simga^2 więc trzeba ją wstaić pod pierwiastek drugiego stopnia )

In [103]:
sigma_a, sigma_T = np.sqrt(np.diag(pcov))   # Δa, ΔT
a_hat, T_hat = popt                         # a, T

In [104]:
print(f"a = {a_hat:.6g} ± {sigma_a:.2g}")
print(f"T = {T_hat:.6g} ± {sigma_T:.2g} [h]")


a = 26383.7 ± 21
T = 67.9101 ± 0.16 [h]


In [105]:
fig, ax = plt.subplots(figsize=(20,12))
ax.plot(t, y, marker='.', label='data')
ax.plot(t, exp_fit(t, *popt), 'r-', label='fit')
ax.set_yscale('log')
ax.grid(True, which='both', alpha=0.3)
ax.legend()

In [106]:
# plt.figure(figsize=(20,12))
# plt.plot(t, y, marker='.', label=cn)
# plt.plot(t, exp_fit(t, *popt), 'r-', label='fit')
# plt.axvline(x= popt[0], color='gray', linestyle='--')
# plt.axvline(x= popt[1], color='gray', linestyle=':')
# plt.grid(True, alpha=0.3)
# plt.legend()
# plt.xlabel('Czas, s')
# plt.ylabel('Liczba zliczen, cps')
# plt.tight_layout()
# plt.show()
# # (opcjonalnie) szybka diagnostyka dopasowania
# res = y - exp_fit(t, *popt)
# print("Parametry (x1, x2_raw, a, b, p, c, T) =", [f"{v:.3g}" for v in popt])
# print(f"RMSE = {np.sqrt(np.mean(res**2)):.6g}, MAE = {np.mean(np.abs(res)):.6g}")


Numer telefonu do tomka

In [107]:
# Bohater
# 791567006

In [108]:
# dane do obliczeń
b_i   = np.array([2.43e-4, 1.363e-3, 1.203e-3, 2.605e-3, 8.19e-4, 1.67e-4])
lambda_i = np.array([1.27e-2, 3.17e-2, 1.15e-1, 3.11e-1, 1.40,   3.87   ])  # [1/s]
B = b_i.sum()  
L = 146e-6


In [109]:
# def rho_T(T_hat, L,B, b_i, lambda_i):
#     suma = np.sum((b_i * B) / (1 + T_hat/lambda_i))
#     nie_suma = L/(T_hat*B)
#     rho = nie_suma + suma
#     return rho
# def sigma_rho_T(sigma_T, L,B, b_i, lambda_i, T_hat):
#     suma = np.sum((b_i * lambda_i) / (B * (lambda_i*T_hat + 1)**2))
#     nie_suma = L/((T_hat**2)*B)
#     sigma_rho = (nie_suma + suma)*sigma_T
#     return sigma_rho
def rho_T(T_hat, L,B, b_i, lambda_i):
    suma = np.sum((b_i) / ((1 /T_hat)+lambda_i))
    nie_suma = 1/(T_hat*B)
    rho = nie_suma * (L+ suma)
    return rho
def sigma_rho_T(sigma_T, L,B, b_i, lambda_i, T_hat):
    suma = np.sum((b_i * lambda_i) / (B * (lambda_i*T_hat + 1)**2))
    nie_suma = L/((T_hat**2)*B)
    sigma_rho = (nie_suma + suma)*sigma_T
    return sigma_rho

In [110]:
# prosta funkcja "±" jak w Twoim użyciu pm()
def pm(val, err, vfmt="{:.2f}", efmt="{:.2f}"):
    return f"{vfmt.format(val)} ± {efmt.format(err)}"

In [111]:
# Załóżmy, że masz:
# T_hat = ...
# sigma_T = ...

r = rho_T(T_hat, L, B, b_i, lambda_i)
dr = sigma_rho_T(sigma_T, L, B, b_i, lambda_i, T_hat)

print("ρ(T) =", r)
print("Δρ(T) =", dr)
print("ρ ± Δρ =", pm(r, dr, vfmt="{:.6f}", efmt="{:.6f}"))
print ("T ± ΔT =", pm(T_hat, sigma_T, vfmt="{:.2f}", efmt="{:.2f}"), "[s]")


ρ(T) = 0.12944096216464201
Δρ(T) = 0.00021483747091936189
ρ ± Δρ = 0.129441 ± 0.000215
T ± ΔT = 67.91 ± 0.16 [s]
